In [ ]:
# default_exp core

# core

> functions to download data.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import numpy as np
import pandas as pd
from fastcore.all import *
from babino2020masks.lasso import *

In [ ]:
#export
def get_bsas_data():
    df = pd.read_csv('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/reporte-covid/dataset_reporte_covid_sitio_gobierno.csv')
    df['Date'] = pd.to_datetime(df.FECHA.map(lambda x:x[:2]+x[2:].capitalize()), format='%d%b%Y:00:00:00')
    df = df.sort_values('Date')
    df = df.loc[df.SUBTIPO_DATO.isin(['personas_hisopadas_reportados_del_dia_totales', '%_positividad_personas_hisopadas_reportadas_del_dia_totales']), ['Date', 'SUBTIPO_DATO', 'VALOR']]
    df = df.set_index(['Date','SUBTIPO_DATO'])['VALOR'].unstack()
    df = df.rename(columns={'personas_hisopadas_reportados_del_dia_totales': 'Tests', '%_positividad_personas_hisopadas_reportadas_del_dia_totales':'p'})
    df.columns.name = ''
    df.p = df.p*0.01
    df['Positives'] = df.p*df.Tests
    df['Negatives'] = df.Tests-df.Positives
    df['Odds'] = df.Positives / df.Negatives
    df = df.reset_index()
    df['date'] = df.Date
    df = df.set_index('date')
    return df

In [ ]:
#export
def get_bsas_data2():
    url = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/reporte-covid/dataset_reporte_covid_sitio_gobierno.xlsx'
    df = pd.read_excel(url)
    df = df.rename(columns={'FECHA': 'Date'}).sort_values('Date')
    positives = df.loc[df.SUBTIPO_DATO.eq('casos_confirmados_reportados_del_dia'), ['Date', 'VALOR']]
    positives = positives.groupby('Date')['VALOR'].sum().reset_index().rename(columns={'VALOR': 'Positives'})
    tests = df.loc[df.SUBTIPO_DATO.eq('testeos_reportados_del_dia_totales'), ['Date', 'VALOR']]
    tests = tests.rename(columns={'VALOR': 'Tests'})
    df = tests.merge(positives, on='Date')
    df['Negatives'] = df.Tests - df.Positives
    df['Odds'] = df.Positives / df.Negatives
    df['p'] = df.Positives / df.Tests
    df['date'] = df.Date
    df = df.set_index('date')
    return df

In [ ]:
#export
@patch
def fit(self:LassoICSelector, X, y, mask=None):
    Xnew = self.transform_to_ols(X)
    if mask is None: mask = np.array([True]*Xnew.shape[1])
    Xnew = Xnew[:, mask]
    self.ols = self.OLS(y, Xnew)
    self.ols_results = self.ols.fit()   
    mask[mask] = (self.ols_results.pvalues < self.alpha / len(self.ols_results.pvalues))
    mask[0] = True
    mask[-1] = True
    if any(self.ols_results.pvalues[1:-1] >= self.alpha / len(self.ols_results.pvalues)): 
        self.fit(X, y, mask=mask)
    self.support = self.selector.get_support()
    self.support[self.support] = mask[:-1]

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
